In [27]:
import minicons.utils
!pip install minicons


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import os
from minicons import cwe
from minicons import scorer

# Load stimuli and model

In [29]:
stimuli_folder = "./stimuli"
stimuli = []
for file in os.listdir(stimuli_folder):
    with open(stimuli_folder + "/" + file, 'r') as stimulus:
        file_contents = stimulus.read().replace('\n', '')
        text_split_at_space = file_contents.split(" ")
        while text_split_at_space.__contains__(""):
            text_split_at_space.remove("")
        stimuli.append(text_split_at_space)

model = cwe.CWE('bert-base-uncased')

# Produce Sliding window predictions

In [34]:
def get_context_words_for_pos(arr: list, pos: int, window_size: int) -> [str]:
    min_index = max(0, pos - (window_size + 1))
    context = arr[min_index]
    for i in range(min_index + 1, pos + 1):
        context = context + " " + arr[i]
    return [context, arr[pos]]


context_size = 5
dataset = stimuli[0]
pos = 2
layer_to_extract = None


representations = []
for pos in range(len(dataset)):
    context_words = get_context_words_for_pos(dataset, pos, context_size)
    representation = model.extract_representation(context_words, layer=layer_to_extract)
    representations.append([context_words, representation])

In [35]:
print(representations[0][1])

tensor([[ 3.8546e-01,  2.5927e-02,  4.5724e-01,  2.4660e-01, -1.8420e-01,
          4.0395e-01,  2.5618e-02,  7.5747e-01, -3.1473e-01, -2.8724e-01,
         -3.5021e-01,  8.0802e-02, -3.5961e-02,  1.4989e-01, -1.2546e-01,
          3.8521e-01, -4.2257e-01, -6.8980e-02, -2.2295e-01, -1.1200e-01,
          5.6545e-02, -2.9890e-01, -3.1384e-01,  4.3110e-01, -4.5344e-01,
          2.6639e-01, -4.8424e-01, -8.1909e-01, -3.2558e-01,  3.4584e-01,
          3.1353e-01,  4.0233e-01, -3.5318e-01, -5.7898e-01, -6.5884e-01,
          1.5276e-01, -2.7353e-01, -4.5331e-01, -2.3076e-01, -9.7108e-02,
          9.5714e-03, -2.4807e-01, -9.1338e-02,  1.1594e-01,  6.1116e-02,
         -4.6243e-01,  3.5154e-01, -1.0457e-01,  5.3796e-02, -4.8131e-01,
         -2.8545e-01,  2.7677e-01, -1.3215e-01, -3.3050e-01, -2.0261e-01,
          6.4922e-01,  1.2876e-01,  2.1384e-01, -1.3975e-01,  1.5448e-01,
         -4.4165e-01,  3.6759e-01,  7.5730e-01,  1.6125e-01,  3.0599e-03,
          1.0378e-01, -7.0599e-02,  3.

# Compute acceptability scores of the words predicted

Todo

In [37]:
s2s_model = scorer.Seq2SeqScorer('t5-base', 'cpu')

# sequence scoring for batch of input, output, by default = logprobs, can change to other quantities as needed (see minicons readme)
s2s_model.conditional_score(["What is the capital of France?", "What is the capital of France?"], ["Paris.", "Lyon."]) # the same thing works with ilm_model and mlm_model as well

'''OUTPUT:
[-6.089522838592529, -8.20227336883545]
''' 

# Token-wise score of the output queries: -- <pad> token is given a score of 0.0, pass rank=True to also give token ranks
s2s_model.conditional_token_score(["What is the capital of France?", "What is the capital of France?"], ["Paris.", "Lyon."], rank=True) 

'''OUTPUT:
[[('<pad>', 0.0, 0),
  ('Paris', -7.5618486404418945, 168),
  ('.', -4.617197036743164, 11)],
 [('<pad>', 0.0, 0),
  ('Lyon', -12.044157981872559, 3459),
  ('.', -4.36038875579834, 8)]]
'''

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

C:\Users\Leo\PycharmProjects\miniconsPlayground\venv\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Leo\.cache\huggingface\hub\models--t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

"OUTPUT:\n[[('<pad>', 0.0, 0),\n  ('Paris', -7.5618486404418945, 168),\n  ('.', -4.617197036743164, 11)],\n [('<pad>', 0.0, 0),\n  ('Lyon', -12.044157981872559, 3459),\n  ('.', -4.36038875579834, 8)]]\n"